## Importação de Bibliotecas

In [1]:
# Importando bibliotecas
import pandas as pd
import os 
import warnings
import folium
import openrouteservice
import ast

In [2]:
#Ajustes
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidt", None)
warnings.filterwarnings("ignore")

## Importando os dados

In [3]:
custumers = pd.read_csv("Dados/custumers.csv", sep=",", encoding="utf-8")
plant = pd.read_csv("Dados/plant.csv", sep=",", encoding="utf-8")



In [4]:
plant

,id_custumer,plant_id,plant_name,Latitude,Longitude
0,2464,SP001,Sorocaba,-23.43386,-47.39249
1,2467,SP001,Sorocaba,-23.43386,-47.39249
2,2488,SP001,Sorocaba,-23.43386,-47.39249
3,2491,SP001,Sorocaba,-23.43386,-47.39249
4,2494,SP001,Sorocaba,-23.43386,-47.39249
...,...,...,...,...,...
430,5272,SP001,Sorocaba,-23.43386,-47.39249
431,5350,SP001,Sorocaba,-23.43386,-47.39249
432,5353,SP001,Sorocaba,-23.43386,-47.39249
433,5356,SP001,Sorocaba,-23.43386,-47.39249


In [5]:
filiais = plant[["plant_name", "Latitude", "Longitude"]]
filiais = filiais[filiais["plant_name"].isin(["Sorocaba"])].drop_duplicates().reset_index()
filiais

,index,plant_name,Latitude,Longitude
0,0,Sorocaba,-23.43386,-47.39249


In [6]:
custumers

,id_custumer,custumer_name,City,Latitude,Longitude
0,2431,CUSTOMER 2431,Alagoinhas,-12.103680,-38.355090
1,2434,CUSTOMER 2434,Alfenas,-21.466445,-45.996509
2,2437,CUSTOMER 2437,Alfenas,-21.466855,-45.984828
3,2440,CUSTOMER 2440,Anápolis,-16.424773,-48.993149
4,2443,CUSTOMER 2443,Anápolis,-16.336820,-48.987640
...,...,...,...,...,...
995,5416,CUSTOMER 5416,Vitória,-20.300200,-40.292200
996,5419,CUSTOMER 5419,Vitória,-20.322200,-40.274200
997,5422,CUSTOMER 5422,Vitória,-20.299803,-40.292414
998,5425,CUSTOMER 5425,Vitória,-20.299906,-40.292401


In [7]:
df = pd.merge(
    plant,
    custumers,
    on="id_custumer",
    how="inner"
)

In [8]:
df.sample(50)

,id_custumer,plant_id,plant_name,Latitude_x,Longitude_x,custumer_name,City,Latitude_y,Longitude_y
100,3274,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3274,Itu,-23.311214,-47.316202
122,3451,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3451,Limeira,-22.607200,-47.400200
131,3478,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3478,Limeira,-22.593972,-47.442184
324,4867,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 4867,São Paulo,-23.591404,-46.674487
220,4093,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 4093,Rio Claro,-22.450587,-47.591959
31,2746,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 2746,Campinas,-22.897937,-47.188105
47,2968,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 2968,Diadema,-23.742004,-46.644499
429,5269,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 5269,Tatuí,-23.393095,-47.912774
176,3874,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3874,Piracicaba,-22.706028,-47.656763
129,3472,SP001,Sorocaba,-23.43386,-47.39249,CUSTOMER 3472,Limeira,-22.584200,-47.412200


In [9]:
#Função para encontrar e retornar latitude e longitude de cada cliente da entrega
def encontrar_long_lat_clientes(lista_clientes, cidade_escolhida):
    #Filtrando o dataframe
    clientes_filtrados = df[df["id_custumer"].isin(lista_clientes)]

    #Ordenando o dataframe na mesma ordem da lista de clientes 
    clientes_filtrados = clientes_filtrados.set_index("id_custumer").reindex(lista_clientes)

    #Retornando lista com tuplas de longitude e latitude de clientes
    long_lat = list(zip(clientes_filtrados["Longitude_y"], clientes_filtrados["Latitude_y"]))

    #Coordenadas de Sorocaba e Campinas
    filiais = {
        "Sorocaba": [(-47.39249,-23.43386)]
    }

    #incluindo itens na lista final, iniciando e terminando na cidade escolhida
    lista_final = filiais[cidade_escolhida]
    lista_final.extend(long_lat)
    lista_final.append(filiais[cidade_escolhida][0])

    return lista_final



In [37]:
#Testando a função
lista_clientes = custumers[custumers["custumer_name"].isin(["CUSTOMER 3496"])]["id_custumer"].tolist()
coordenadas = encontrar_long_lat_clientes(lista_clientes, "Sorocaba")
coordenadas

[(-47.39249, -23.43386), (-46.676315, -22.550753), (-47.39249, -23.43386)]

In [38]:

def calculando_rota(coordenadas):
    # Configurar chave API
    ORS_API_KEY = os.getenv("TOKEN_API") #Busca a API

    # Criar cliente OpenRouteService
    client = openrouteservice.Client(key=ORS_API_KEY)

    try:
        # Exemplo: calcular rota com as coordenadas já obtidas
        rota = client.directions(
            coordinates=coordenadas,
            profile="driving-car", 
            # profile="driving-hgv", 
            format_out="geojson"    
        )

        return rota
    
    except: 
        return "N/A"
 

In [39]:
rota = calculando_rota(coordenadas)

if rota is None:
    print("Falha ao calcular rota. Verifique coordenadas.")
else:
    distancia_total = 0
    duracao_total = 0
    for segment in rota["features"][0]["properties"]["segments"]:
        distancia_total += segment["distance"]
        duracao_total += segment["duration"]

    distancia_total_km = distancia_total / 1000
    duracao_total_min = duracao_total / 60

    print(f"Distância: {distancia_total_km:.2f} km")
    print(f"Duração: {duracao_total_min:.1f} minutos")


Distância: 365.89 km
Duração: 278.8 minutos


In [41]:
# Exemplo de entragas/Clientes 
lista_clientes = lista_clientes

# Buscando Coordenadas (retorna no formato ORS: (lon, lat))
coord_sorocaba = encontrar_long_lat_clientes(lista_clientes, "Sorocaba")

# Calculando rotas
rota_sorocaba = calculando_rota(coord_sorocaba)

# Criando o objeto mapa — convertendo (lon, lat) -> (lat, lon)
lat = coord_sorocaba[1][1]
lon = coord_sorocaba[1][0]

mapa = folium.Map(location=[lat, lon], zoom_start=10)

# Plotando rota
folium.GeoJson(
    rota_sorocaba, 
    name="Rota Ideal", 
    style_function=lambda x: {
        "color": "red",
        "weight": 4,
    }
).add_to(mapa)

# Adicionando marcadores
for lon, lat in coord_sorocaba:
    folium.Marker(
        location=[lat, lon],    # Folium = (lat, lon)
        icon=folium.Icon(color="red", icon="flag")
    ).add_to(mapa)

mapa.save("mapa.html")